<a href="https://colab.research.google.com/github/Sohamghosh98/Siksha/blob/main/Shiksha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow pandas numpy scikit-learn

In [20]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import MinMaxScaler


In [15]:

from google.colab import files
uploaded = files.upload()

Saving shiksha.csv to shiksha.csv


In [17]:
shiksha=pd.read_csv("shiksha.csv")
shiksha

,student_id,question_id,correct,time_taken,attempts,hint_used,difficulty
0,S001,Q1,1,14,1,0,2
1,S001,Q2,1,18,1,0,2
2,S001,Q3,0,32,2,1,3
3,S001,Q4,1,16,1,0,2
4,S001,Q5,1,15,1,0,1
5,S002,Q1,0,40,3,1,3
6,S002,Q2,1,28,2,0,2
7,S002,Q3,0,45,3,1,3
8,S002,Q4,1,25,2,0,2
9,S002,Q5,0,38,3,1,3


In [18]:
shiksha.info()
shiksha.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   student_id   20 non-null     object
 1   question_id  20 non-null     object
 2   correct      20 non-null     int64 
 3   time_taken   20 non-null     int64 
 4   attempts     20 non-null     int64 
 5   hint_used    20 non-null     int64 
 6   difficulty   20 non-null     int64 
dtypes: int64(5), object(2)
memory usage: 1.2+ KB


,0
student_id,0
question_id,0
correct,0
time_taken,0
attempts,0
hint_used,0
difficulty,0


In [19]:
features = (
    shiksha
    .groupby("student_id")
    .apply(lambda x: pd.Series({
        "accuracy": x["correct"].mean(),
        "avg_time": x["time_taken"].mean(),
        "avg_attempts": x["attempts"].mean(),
        "hint_ratio": x["hint_used"].mean(),
        "difficulty_score": (x["correct"] * x["difficulty"]).sum() / x["difficulty"].sum()
    }))
    .reset_index()
)

features


/tmp/ipython-input-3915809130.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series({


,student_id,accuracy,avg_time,avg_attempts,hint_ratio,difficulty_score
0,S001,0.8,19.0,1.2,0.2,0.700000
1,S002,0.4,35.2,2.6,0.6,0.307692
2,S003,1.0,14.0,1.0,0.0,1.000000
3,S004,0.2,51.6,2.8,0.8,0.142857


In [21]:
scaler = MinMaxScaler()
cols_to_scale = ["accuracy", "avg_time", "avg_attempts", "hint_ratio", "difficulty_score"]

features_scaled = features.copy()
features_scaled[cols_to_scale] = scaler.fit_transform(features[cols_to_scale])

features_scaled

,student_id,accuracy,avg_time,avg_attempts,hint_ratio,difficulty_score
0,S001,0.75,0.132979,0.111111,0.25,0.650000
1,S002,0.25,0.563830,0.888889,0.75,0.192308
2,S003,1.00,0.000000,0.000000,0.00,1.000000
3,S004,0.00,1.000000,1.000000,1.00,0.000000


In [22]:
def assign_label(row):
    if row["accuracy"] >= 0.75 and row["hint_ratio"] <= 0.2:
        return 2   # Strong
    elif row["accuracy"] >= 0.45:
        return 1   # Average
    else:
        return 0   # Needs Help

features_scaled["label"] = features_scaled.apply(assign_label, axis=1)
features_scaled

,student_id,accuracy,avg_time,avg_attempts,hint_ratio,difficulty_score,label
0,S001,0.75,0.132979,0.111111,0.25,0.650000,1
1,S002,0.25,0.563830,0.888889,0.75,0.192308,0
2,S003,1.00,0.000000,0.000000,0.00,1.000000,2
3,S004,0.00,1.000000,1.000000,1.00,0.000000,0


In [24]:
X = features_scaled.drop(["student_id", "label"], axis=1).values
y = features_scaled["label"].values

X.shape, y.shape


((4, 5), (4,))

In [28]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42
)

X_train.shape, X_test.shape

((3, 5), (1, 5))

In [29]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation="relu", input_shape=(X.shape[1],)),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3, activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 32)             │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 771 (3.01 KB)

 Trainable params: 771 (3.01 KB)

 Non-trainable params: 0 (0.00 B)

In [30]:
history = model.fit(
    X_train,
    y_train,
    epochs=40,
    batch_size=4,
    validation_data=(X_test, y_test),
    verbose=1
)


Epoch 1/40
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.3333 - loss: 1.1769 - val_accuracy: 0.0000e+00 - val_loss: 1.2905
Epoch 2/40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - accuracy: 0.3333 - loss: 1.1619 - val_accuracy: 0.0000e+00 - val_loss: 1.2846
Epoch 3/40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 0.3333 - loss: 1.1248 - val_accuracy: 0.0000e+00 - val_loss: 1.2806
Epoch 4/40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 0.3333 - loss: 1.0846 - val_accuracy: 0.0000e+00 - val_loss: 1.2766
Epoch 5/40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - accuracy: 0.3333 - loss: 1.0557 - val_accuracy: 0.0000e+00 - val_loss: 1.2722
Epoch 6/40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - accuracy: 0.3333 - loss: 1.1223 - val_accuracy: 0.0000e+00 - val_loss: 1.2671
Epoch 7/40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - accuracy: 0.3333 - loss: 1.0836 - val_accuracy: 0.0000e+00 - val_loss: 1.2624
Epoch 8/40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - accuracy: 0.3333 - loss: 1.0925 - val_acc

In [31]:
y_pred = np.argmax(model.predict(X_test), axis=1)

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
confusion_matrix(y_test, y_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       1.0
           1       0.00      0.00      0.00       0.0

    accuracy                           0.00       1.0
   macro avg       0.00      0.00      0.00       1.0
weighted avg       0.00      0.00      0.00       1.0

Confusion Matrix:


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_

array([[0, 1],
       [0, 0]])

In [32]:
model.save("shikshasathi_stage1_model.h5")


In [33]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [34]:
features_scaled.to_csv("shikshasathi_features_stage1.csv", index=False)

In [35]:
# Load dataset
df = pd.read_csv("shikshasathi_features_stage1.csv")

X = df.drop(["student_id", "label"], axis=1).values
y = df["label"].values

# Load trained model
model = tf.keras.models.load_model("shikshasathi_stage1_model.h5")

In [36]:
y_prob = model.predict(X)
y_pred = np.argmax(y_prob, axis=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step


In [37]:
accuracy = accuracy_score(y, y_pred)
accuracy

0.25

In [39]:
labels = ["Needs Help", "Average", "Strong"]
cm_df = pd.DataFrame(cm, index=labels, columns=labels)
cm_df

,Needs Help,Average,Strong
Needs Help,0,2,0
Average,0,0,1
Strong,0,0,1


In [40]:
print(classification_report(
    y, y_pred,
    target_names=["Needs Help", "Average", "Strong"]
))

              precision    recall  f1-score   support

  Needs Help       0.00      0.00      0.00         2
     Average       0.00      0.00      0.00         1
      Strong       0.50      1.00      0.67         1

    accuracy                           0.25         4
   macro avg       0.17      0.33      0.22         4
weighted avg       0.12      0.25      0.17         4



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [41]:
errors = df.copy()
errors["predicted"] = y_pred
errors[errors["label"] != errors["predicted"]]

,student_id,accuracy,avg_time,avg_attempts,hint_ratio,difficulty_score,label,predicted
0,S001,0.75,0.132979,0.111111,0.25,0.650000,1,2
1,S002,0.25,0.563830,0.888889,0.75,0.192308,0,1
3,S004,0.00,1.000000,1.000000,1.00,0.000000,0,1


In [42]:
model = tf.keras.models.load_model("shikshasathi_stage1_model.h5")
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 32)             │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 773 (3.02 KB)

 Trainable params: 771 (3.01 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [43]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Enable optimization (important for demo + offline use)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()

Saved artifact at '/tmp/tmpaao5ul5w'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 5), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  139376766772112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139376766774800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139376766771344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139376766774608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139376766774224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139376766775184: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [44]:
with open("shikshasathi_stage1_model.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ TFLite model saved successfully")

✅ TFLite model saved successfully


In [45]:
from google.colab import files
files.download("shikshasathi_stage1_model.tflite")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [46]:
interpreter = tf.lite.Interpreter(model_path="shikshasathi_stage1_model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_details, output_details

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


([{'name': 'serving_default_input_layer_1:0',
   'index': 0,
   'shape': array([1, 5], dtype=int32),
   'shape_signature': array([-1,  5], dtype=int32),
   'dtype': numpy.float32,
   'quantization': (0.0, 0),
   'quantization_parameters': {'scales': array([], dtype=float32),
    'zero_points': array([], dtype=int32),
    'quantized_dimension': 0},
   'sparsity_parameters': {}}],
 [{'name': 'StatefulPartitionedCall_1:0',
   'index': 10,
   'shape': array([1, 3], dtype=int32),
   'shape_signature': array([-1,  3], dtype=int32),
   'dtype': numpy.float32,
   'quantization': (0.0, 0),
   'quantization_parameters': {'scales': array([], dtype=float32),
    'zero_points': array([], dtype=int32),
    'quantized_dimension': 0},
   'sparsity_parameters': {}}])

In [47]:
def predict_with_tflite(features):
    input_data = np.array([features], dtype=np.float32)

    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()

    output = interpreter.get_tensor(output_details[0]['index'])
    return np.argmax(output), output

In [48]:
sample_student = [0.82, 0.30, 0.25, 0.10, 0.80]  # normalized values

pred_class, probs = predict_with_tflite(sample_student)

mapping = {
    0: "Needs Help",
    1: "Average",
    2: "Strong"
}

print("Predicted Level:", mapping[pred_class])
print("Probabilities:", probs)

Predicted Level: Strong
Probabilities: [[0.22825444 0.3289312  0.44281435]]


In [50]:
raw_df = pd.read_csv("shiksha.csv")
raw_df.head()

,student_id,question_id,correct,time_taken,attempts,hint_used,difficulty
0,S001,Q1,1,14,1,0,2
1,S001,Q2,1,18,1,0,2
2,S001,Q3,0,32,2,1,3
3,S001,Q4,1,16,1,0,2
4,S001,Q5,1,15,1,0,1


In [52]:
student_features = (
    raw_df
    .groupby("student_id")
    .apply(lambda x: pd.Series({
        "accuracy": x["correct"].mean(),
        "avg_time": x["time_taken"].mean(),
        "avg_attempts": x["attempts"].mean(),
        "hint_ratio": x["hint_used"].mean(),
        "difficulty_score": (x["correct"] * x["difficulty"]).sum() / x["difficulty"].sum()
    }))
    .reset_index()
)

student_features

/tmp/ipython-input-1827749263.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series({


,student_id,accuracy,avg_time,avg_attempts,hint_ratio,difficulty_score
0,S001,0.8,19.0,1.2,0.2,0.700000
1,S002,0.4,35.2,2.6,0.6,0.307692
2,S003,1.0,14.0,1.0,0.0,1.000000
3,S004,0.2,51.6,2.8,0.8,0.142857


In [53]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
feature_cols = ["accuracy", "avg_time", "avg_attempts", "hint_ratio", "difficulty_score"]

student_features_scaled = student_features.copy()
student_features_scaled[feature_cols] = scaler.fit_transform(
    student_features[feature_cols]
)

student_features_scaled


,student_id,accuracy,avg_time,avg_attempts,hint_ratio,difficulty_score
0,S001,0.75,0.132979,0.111111,0.25,0.650000
1,S002,0.25,0.563830,0.888889,0.75,0.192308
2,S003,1.00,0.000000,0.000000,0.00,1.000000
3,S004,0.00,1.000000,1.000000,1.00,0.000000


In [54]:
import tensorflow as tf
import numpy as np

interpreter = tf.lite.Interpreter(model_path="shikshasathi_stage1_model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


In [55]:
def tflite_predict(features_array):
    input_data = features_array.astype(np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])
    return output


In [57]:
X = student_features_scaled[feature_cols].values

all_probs = []
for i in range(X.shape[0]):
    # Reshape the single sample to be (1, 5) as expected by the TFLite model
    sample_features = X[i:i+1]
    all_probs.append(tflite_predict(sample_features))

probs = np.vstack(all_probs)

predicted_class = np.argmax(probs, axis=1)
confidence = np.max(probs, axis=1)

In [58]:
label_map = {
    0: "Needs Help",
    1: "Average",
    2: "Strong"
}

student_features_scaled["predicted_level"] = [
    label_map[i] for i in predicted_class
]

student_features_scaled["confidence"] = confidence.round(2)


In [59]:
def recommendation(row):
    if row["predicted_level"] == "Needs Help":
        return "Provide remedial content and teacher intervention"
    elif row["predicted_level"] == "Average":
        return "Regular practice with guided support"
    else:
        return "Advanced problems and peer mentoring"

student_features_scaled["recommendation"] = student_features_scaled.apply(
    recommendation, axis=1
)


In [60]:
final_report = student_features_scaled[
    ["student_id", "predicted_level", "confidence", "recommendation"]
]

final_report


,student_id,predicted_level,confidence,recommendation
0,S001,Strong,0.42,Advanced problems and peer mentoring
1,S002,Average,0.35,Regular practice with guided support
2,S003,Strong,0.49,Advanced problems and peer mentoring
3,S004,Average,0.35,Regular practice with guided support


In [61]:
final_report.to_csv("student_wise_analysis.csv", index=False)


In [62]:
from google.colab import files
files.download("student_wise_analysis.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>